<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Github_Repos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %pip install GitPython

In [ ]:
from __future__ import absolute_import
from IPython.display import clear_output
import git, os, pathlib
from google.colab import drive, files
drive.mount("/gdrive", force_remount=True)
os.chdir("/gdrive/My Drive")
print(pathlib.Path.cwd())

import BashColors
from BashColors import C
import FileUtilities
from FileUtilities import *
fu.listFiles(silent=True)

In [ ]:
%%writefile GithubRepos.py
# 09/10/2020
import git, os
from os.path import *
from BashColors import C

class GithubRepos:
        
    def __init__(self):
        self.gr = GithubRepos
        self.repoList = []
        
    def getCurrentProjectRepo(self):
        currentProjectURL = 'https://github.com/iypc-team/CurrentProject'
        
        localURL = os.path.join(os.curdir, basename(currentProjectURL))
        print(basename(currentProjectURL))
        if not exists(localURL):
            currentProjectRepo = git.Repo.clone_from(currentProjectURL, localURL)
            self.repoList.append(currentProjectRepo)
        else: print(f'{C.HEADER}{basename(localURL)} already exists')
    
    def getCoLabRepo(self):
        thisRepo = None
        colabURL = 'https://github.com/iypc-team/CoLab'
        self.updateRepoList(name=colabURL)
        localURL = os.path.join(os.path.curdir, basename(colabURL))
        if not exists(localURL):
            thisRepo = git.Repo.clone_from(colabURL, localURL)
            self.repoList.append(thisRepo)
            print(f'{C.Green}{thisRepo}')
        else: print(f'{C.Red}{thisRepo}')

    def listRemotes(self):
        print(f'{C.Blue}Remotes:')
        for rpo in self.repoList:
            print(rpo)
            print(f'{rpo}')

    def updateRepoList(self, name):
        if not name in self.repoList:
            self.repoList.append(name)
            
gr = GithubRepos()

In [ ]:
import GithubRepos
from GithubRepos import *

In [ ]:

# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import datetime, time
from time import strftime, strptime

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "title contains '*.*' and 'root' in parents"}).GetList()
fil = None
for fil in listed:
    print(F'')
    fileId = fil['id']
    title = fil['title']
    kind = fil['kind']
    labels = fil['labels'] # Returns a dictionary
    modified = fil['modifiedDate']

    mType = fil['mimeType']

    strfTime = strftime(modified)

    created = fil['createdDate']

    print(f'{C.White}title:    {C.IBlue}{title}\t{kind}{C.IWhite}')
    print(f'{C.White}mimeType: {C.Cyan}{mType}')
    print(f'{C.White}fileId:   {C.White}{fileId}')
    print(f'{C.White}created:  {C.Cyan}{C.BGreen}{created}{C.IWhite}')
    print(f'{C.White}modified: {C.Cyan}{C.IYellow}{modified}{C.IWhite}')
    print(f'{C.White}strfTime: {C.IPurple}{strfTime}{C.IWhite}')
    print(f'\n')

In [ ]:

for gfil in listed:
    gDate = gfil['modifiedDate']
    gType = gfil['mimeType']
    realPath = realpath(gfil['title'])
    if exists(realPath) and isdir(realPath) and not realPath.__contains__('__'):
        stat1 = os.stat(realPath).st_mtime
        tziStat = tzi.getFileStats(realPath)
        print(f'{C.BIBlue}{realPath}{C.White}')
        print(gType)
        print(stat1) #  floating point number 
        print(gDate) #  string 
        print(tziStat) #  string 
        print()

In [ ]:
aboutDict = drive.GetAbout() # [str:str]

for item in aboutDict:
    value = aboutDict[item]
    print(f'{item}: {value}')

In [ ]:
wtf = drive.ListFile().GetList()
myFilesDict = {}
filCount = 0
for item in wtf:
    title = item['title']
    fileId = item['id']
    trying = item['embedLink']

    if title.__contains__('.pyc'):
        pass
    elif title.__contains__('.'):
        filCount += 1
        realPath = realpath(title)
        myFilesDict[filCount] = (fileId, title, realPath)

        print(f'{C.White}{filCount}. {C.Green}{title}')
        print(f'{C.White}fileId: {C.Purple}{fileId}')
        print()

In [ ]:
for number in myFilesDict:
    result = myFilesDict[number]
    print(f'{number}. {result[1]}')

In [ ]:
def getGoogleDriveFileId(filID, silent=True):
    # Download a file based on its file ID.
    file_id = fileId
    downloaded = drive.CreateFile({'id': file_id})
    if not silent:
        content = downloaded.GetContentString()
        print(f'{content}')
        # print('Downloaded content "{}"'.format(downloaded.GetContentString()))
    else:
        return downloaded.GetContentString()

In [ ]:
pth = myFilesDict[6]
print(f'{C.White}pth[0]: {C.Green}{pth[0]}')
print(f'{C.White}pth[1]: {C.Blue}{pth[1]}')
print(f'{C.White}pth[2]: {C.Blue}{pth[2]}{C.ColorOff}')

fil_id = pth[0]
dlFile = getGoogleDriveFileId(fil_id)
print(dlFile)

In [ ]:

deleteFilesDict = {}
for itemNumber in fu.allFilesDict:
    pth = fu.allFilesDict[itemNumber]
    realPth = realpath(pth)
    if pth.name.__contains__('GDriveFileUtilities') or realPth.__contains__('GDrive_'):
        deleteFilesDict[itemNumber] = basename(realPth)
        print(f'{C.IWhite}{itemNumber}. {C.IRed}{realPth}')
print(f'{C.ColorOff}')

for itemNumber in deleteFilesDict:
    fileName = deleteFilesDict[itemNumber]
    print(deleteFilesDict[itemNumber])
    getGoogleDriveFileId(filID=fileName, silent=False)

In [ ]:
def updateGoogleDriveFileIDs(extension='*'):
    def parseDateTime(strng):
        # print(strng)
        tIndex = strng.rfind('T')
        dotIndex = strng.rfind('.')
        date_str = strng[0:tIndex]
        time_str = strng[tIndex + 1:dotIndex]
        newString = date_str + '  ' + time_str
        # print(newString)
        return newString

    gdriveFiles = drive.ListFile().GetList()
    gdriveFilesDict = {}
    counter = 0
    for fil in gdriveFiles:
        fileId = fil['id']
        fileName = fil['title']
        fileStats = parseDateTime(fil['modifiedDate'])
        # parseDateTime(fileStats)
        if fileName.endswith('.pyc'):
            pass
        else:
            thisPath = realpath(fileName)
            counter += 1
            gdriveFilesDict[fileName] = fileId
            print(f'{C.IWhite}{counter}. {C.Green}{fileName}. {C.ICyan}{fileStats}')
            print(f'{C.IRed}{fileId}\n')
updateGoogleDriveFileIDs(extension='py')